<a href="https://colab.research.google.com/github/aidan-sc/WDCNN_classifier/blob/main/WDCNN_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Data preprocessing

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os
print(f"cwd:{os.getcwd()}")

if os.getcwd() != 'content/drive/MyDrive/paderborn':
  os.chdir('./drive/MyDrive/paderborn')

In [ ]:
import time
startTime = time.time()

In [ ]:
import numpy as np

from paderborn_data_loader import PaderbornData

In [ ]:
!pip3 install keras-visualizer
from keras_visualizer import visualizer

### Split the Paderborn bearing data into training and testing sets

In [ ]:
root_dir = './data/raw/'
experiment = PaderbornData(root_dir, experiment='artificial', datastream='vibration', normalisation='robust-zscore')
x_train, y_train, x_test, y_test = experiment.split_data(250000,
                                                         train_fraction=0.5,
                                                         window_step=1024,
                                                         window_length=4500,
                                                         verbose=False)

In [ ]:
print(x_train.shape)
print(y_train.shape)

## Train our WDCNN model

### First split the data, scale it, and convert labels to one hot encoding

In [ ]:
import pandas as pd
import numpy as np
import os

import matplotlib.pyplot as plt

import tensorflow as tf

In [ ]:
from tensorflow.keras.utils import to_categorical

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

### Build our WDCNN model

In [ ]:
from tensorflow.keras.models import Model

from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import MaxPooling1D

from tensorflow.keras.layers import Dropout

# build the wdcnn model
def generate_model(n_class, n_timesteps, n_variables, first_kernel=64, strides=16):

    # set up the shape of the input
    ip = Input(shape=(n_timesteps, n_variables))

    # convolutional layers
    y = Conv1D(16, (first_kernel), strides=strides, padding='same')(ip)
    y = Activation('relu')(y)
    y = BatchNormalization()(y)
    y = MaxPooling1D(2, strides=2, padding='same')(y)

    y = Conv1D(32, (3), padding='same')(y)
    y = Activation('relu')(y)
    y = BatchNormalization()(y)
    y = MaxPooling1D(2, strides=2, padding='same')(y)

    y = Conv1D(64, (3), padding='same')(y)
    y = Activation('relu')(y)
    y = BatchNormalization()(y)
    y = MaxPooling1D(2, strides=2, padding='same')(y)

    y = Conv1D(64, (3), padding='same')(y)
    y = Activation('relu')(y)
    y = BatchNormalization()(y)
    y = MaxPooling1D(2, strides=2, padding='same')(y)

    y = Conv1D(64, (3), padding='same')(y)
    y = Activation('relu')(y)
    y = BatchNormalization()(y)
    y = MaxPooling1D(2, strides=2, padding='same')(y)

    # flatten
    y = Flatten()(y)

    # dense
    y = Dense(100)(y)
    y = BatchNormalization()(y)

    # add the softmax classification outpuy
    out = Dense(n_class, activation='softmax')(y)

    # join the input and the output and return the model
    model = Model(ip, out)

   # visualizer(model, file_name='graph01', file_format='png', view=True)
    return model


In [ ]:
model = generate_model(3, x_train.shape[1], x_train.shape[2], first_kernel=256, strides=8)

In [ ]:
import numpy as np

import keras.backend as K

from keras.callbacks import Callback

class CyclicLR(Callback):
    """This callback implements a cyclical learning rate policy (CLR).
    The method cycles the learning rate between two boundaries with
    some constant frequency, as detailed in this paper (https://arxiv.org/abs/1506.01186).
    The amplitude of the cycle can be scaled on a per-iteration or
    per-cycle basis.
    This class has three built-in policies, as put forth in the paper.
    "triangular":
        A basic triangular cycle w/ no amplitude scaling.
    "triangular2":
        A basic triangular cycle that scales initial amplitude by half each cycle.
    "exp_range":
        A cycle that scales initial amplitude by gamma**(cycle iterations) at each
        cycle iteration.
    For more detail, please see paper.

    # Example
        ```python
            clr = CyclicLR(base_lr=0.001, max_lr=0.006,
                                step_size=2000., mode='triangular')
            model.fit(X_train, Y_train, callbacks=[clr])
        ```

    Class also supports custom scaling functions:
        ```python
            clr_fn = lambda x: 0.5*(1+np.sin(x*np.pi/2.))
            clr = CyclicLR(base_lr=0.001, max_lr=0.006,
                                step_size=2000., scale_fn=clr_fn,
                                scale_mode='cycle')
            model.fit(X_train, Y_train, callbacks=[clr])
        ```
    # Arguments
        base_lr: initial learning rate which is the
            lower boundary in the cycle.
        max_lr: upper boundary in the cycle. Functionally,
            it defines the cycle amplitude (max_lr - base_lr).
            The lr at any cycle is the sum of base_lr
            and some scaling of the amplitude; therefore
            max_lr may not actually be reached depending on
            scaling function.
        step_size: number of training iterations per
            half cycle. Authors suggest setting step_size
            2-8 x training iterations in epoch.
        mode: one of {triangular, triangular2, exp_range}.
            Default 'triangular'.
            Values correspond to policies detailed above.
            If scale_fn is not None, this argument is ignored.
        gamma: constant in 'exp_range' scaling function:
            gamma**(cycle iterations)
        scale_fn: Custom scaling policy defined by a single
            argument lambda function, where
            0 <= scale_fn(x) <= 1 for all x >= 0.
            mode paramater is ignored
        scale_mode: {'cycle', 'iterations'}.
            Defines whether scale_fn is evaluated on
            cycle number or cycle iterations (training
            iterations since start of cycle). Default is 'cycle'.
    """

    def __init__(self, base_lr=0.001, max_lr=0.006, step_size=2000., mode='triangular',
                 gamma=1., scale_fn=None, scale_mode='cycle'):
        super(CyclicLR, self).__init__()

        self.base_lr = base_lr
        self.max_lr = max_lr
        self.step_size = step_size
        self.mode = mode
        self.gamma = gamma
        if scale_fn == None:
            if self.mode == 'triangular':
                self.scale_fn = lambda x: 1.
                self.scale_mode = 'cycle'
            elif self.mode == 'triangular2':
                self.scale_fn = lambda x: 1/(2.**(x-1))
                self.scale_mode = 'cycle'
            elif self.mode == 'exp_range':
                self.scale_fn = lambda x: gamma**(x)
                self.scale_mode = 'iterations'
        else:
            self.scale_fn = scale_fn
            self.scale_mode = scale_mode
        self.clr_iterations = 0.
        self.trn_iterations = 0.
        self.history = {}

        self._reset()

    def _reset(self, new_base_lr=None, new_max_lr=None,
               new_step_size=None):
        """Resets cycle iterations.
        Optional boundary/step size adjustment.
        """
        if new_base_lr != None:
            self.base_lr = new_base_lr
        if new_max_lr != None:
            self.max_lr = new_max_lr
        if new_step_size != None:
            self.step_size = new_step_size
        self.clr_iterations = 0.

    def clr(self):
        cycle = np.floor(1+self.clr_iterations/(2*self.step_size))
        x = np.abs(self.clr_iterations/self.step_size - 2*cycle + 1)
        if self.scale_mode == 'cycle':
            return self.base_lr + (self.max_lr-self.base_lr)*np.maximum(0, (1-x))*self.scale_fn(cycle)
        else:
            return self.base_lr + (self.max_lr-self.base_lr)*np.maximum(0, (1-x))*self.scale_fn(self.clr_iterations)

    def on_train_begin(self, logs={}):
        logs = logs or {}

        if self.clr_iterations == 0:
            K.set_value(self.model.optimizer.lr, self.base_lr)
        else:
            K.set_value(self.model.optimizer.lr, self.clr())

    def on_batch_end(self, epoch, logs=None):

        logs = logs or {}
        self.trn_iterations += 1
        self.clr_iterations += 1

        self.history.setdefault('lr', []).append(K.get_value(self.model.optimizer.lr))
        self.history.setdefault('iterations', []).append(self.trn_iterations)

        for k, v in logs.items():
            self.history.setdefault(k, []).append(v)

        K.set_value(self.model.optimizer.lr, self.clr())


In [ ]:
# from keras.optimizers import SGD, Adam

# # (re)-initialise the optimiser and compile the model
# opt = SGD(lr=0.0, momentum=0.9, nesterov=True)
# clr = CyclicLR(
#   mode='triangular2',
#   base_lr=1e-5,
#   max_lr=1e-3,
#   step_size=8800
# )
# clr._reset()
# model.compile(loss='categorical_crossentropy',
#               optimizer=opt, metrics=['acc'])

### Train the model

In [ ]:
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics='acc')

In [ ]:
val_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(64)
history = model.fit(x_train, y_train, batch_size=64, epochs=10, shuffle=True, validation_data=val_dataset)

### Plot some results

In [ ]:
# summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
loss, acc = model.evaluate(val_dataset)
print("loss: %.2f" % loss)
print("acc:  %.2f" % acc)

### Get the confusion matrix to see what we struggle with

In [ ]:
"""
visualisation_utils.py

make pretty graphs to show classifier performance

(most of these are based on the really useful examples from the
scikit learn user guides!)

author:     alex shenfield
date:       27/04/2018
"""

# numpy is needed for everything :)
import numpy as np
import matplotlib.pyplot as plt

# utilities for managing the data
import itertools

# data analysis functions from scikit learn
from sklearn.metrics import confusion_matrix


# get the classes and actually plot the confusion matrix
def plot_confusion_matrix(y_true, y_pred):

    cm = confusion_matrix(y_true, y_pred)
    classes = np.unique(y_true)
    plot_cm(cm, classes=classes, title=None)


# define a function for plotting a confusion matrix
def plot_cm(cm,
            classes,
            normalize=False,
            title='Confusion matrix',
            cmap=plt.cm.Blues):

    # should we normalise the confusion matrix?
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print('Confusion matrix, with normalization')
    else:
        print('Confusion matrix, without normalization')

    # display in command windows
    print(cm)

    # create a plot for the confusion matrix
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    # if we want a title displayed
    if title:
        plt.title(title)

    fmt = '.3f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    #plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
predictions = model.predict(val_dataset)
print(predictions.shape)

In [ ]:
y_true = np.argmax(y_test, axis=1)
y_pred = np.argmax(predictions, axis=1)
print(y_true.shape)
print(y_pred.shape)

In [ ]:
plot_confusion_matrix(y_true, y_pred)

In [ ]:
endTime = time.time()

print(endTime - startTime)